In [ ]:
from os import listdir
from os.path import isfile, join
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import nltk
from nltk.tokenize import word_tokenize
import pickle
from gensim.similarities import MatrixSimilarity
nltk.download('punkt')

In [ ]:
data = ['oi']
file = open("test/1.txt","r")
data.append(file.read())
file.close()

In [ ]:

data

In [ ]:
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]

In [ ]:
tagged_data

In [ ]:
for i, _d in enumerate(data):
    print(_d)


In [ ]:
with open('label_content_2017_1', 'rb') as f:
    lc_2017_1 = pickle.load(f)

with open('label_content_2017_2', 'rb') as f:
    lc_2017_2 = pickle.load(f)
    
with open('label_content_2018_1', 'rb') as f:
    lc_2018_1 = pickle.load(f)

In [ ]:
def merge_dicts(*dict_args):
    result = {}
    for files in dict_args:
        result.update(files)
    return result

labelContent = merge_dicts(lc_2017_1,lc_2017_2,lc_2018_1)

In [ ]:
tagged_data = [TaggedDocument(words=word_tokenize(content.lower()), tags=[str(label)]) for label, content in labelContent.items()]


In [ ]:
tagged_data

In [ ]:
max_epochs = 100
vec_size = 100#neurons
alpha = 0.025
window = 5

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=3,
                dm =1,
                window = window)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    #print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

In [ ]:
model= Doc2Vec.load("d2v.model")
#to find the vector of a document which is not in training data
test_data = word_tokenize("O verdadeiro conhecimento vem de dentro.".lower())
v1 = model.infer_vector(test_data)
#print(v1)
#print(model.wv.most_similar(positive=[v1]))


sims = model.docvecs.most_similar(positive=[v1])
print(sims)

#print(sims)




# to find most similar doc using tags
similar_doc = model.docvecs.most_similar(1)

for i in range(0,len(tagged_data)):
    if(model.docvecs.most_similar(i)[0][1] > 0.7):
        print('----------------')
        print(tagged_data[i][1])#print label
        print(model.docvecs.most_similar(i)[0])#higher value
        print('----------------')
        print('----------------')

# to find vector of doc in training data using tags or in other words, printing the vector of document at index 1 in training data
#print(model.docvecs['1'])


In [ ]:
labelContent